In [1]:
import pandas as analytics
import numpy as maths
import os
from data_extraction import df_data , df_users , df_items
import warnings
warnings.filterwarnings("ignore")

In [2]:
users = df_data.groupby('user_id').agg({'movie_id':lambda x:x.count()}).sort_values('movie_id').iloc[:20].reset_index()['user_id'].to_list()
users

[36,
 34,
 926,
 873,
 418,
 812,
 809,
 364,
 895,
 888,
 571,
 558,
 824,
 636,
 93,
 866,
 572,
 202,
 631,
 242]

In [3]:
all_genres = df_items.columns[2:]
all_genres

Index(['Action', 'Adventure', 'Animation', 'Children's', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'],
      dtype='object')

In [4]:
def find_interesting(df_user):
    df_interests = df_user[df_user.columns[2:]].value_counts(normalize = True).reset_index().sort_values('proportion',ascending = False).drop('proportion',axis = 1)
    df_interests = df_interests.iloc[:5]
    
    df_interesting = analytics.DataFrame()
    
    for row in range(len(df_interests)):
        df_temp = df_items[df_items[all_genres[0]] == df_interests.iloc[row][all_genres[0]]]
        for genre in all_genres :
            df_temp = df_temp[df_items[genre] == df_interests.iloc[row][genre]]
        df_interesting = analytics.concat([df_interesting, df_temp])
    
    df_interesting = df_interesting.drop_duplicates()
    return df_interesting

In [5]:
N_values = [10,15,20]
# N_values = [10]

N_counts = {}

for N in N_values :
    print("\nFor",N)
    hit_rates = []
    for user in users :
        hit_rate = 0
        df_user = df_data[df_data['user_id'] == user].sort_values('timestamp')
        df_user = df_user.merge(df_items, on = 'movie_id').drop(['user_id','rating','timestamp'], axis = 1)
        df_interesting = find_interesting(df_user)
        print("\n\tUser #",users.index(user), "User ID :",user)
        for count in range(len(df_user)):
            df_selected = df_interesting.sample( n = N)
            df_reacted = df_selected[df_selected['movie_id'].isin(df_user['movie_id'])]
            if len(df_reacted) > 0 : 
                f = 1
                df_reacted = df_reacted.sample(n = 1)
            else : f = 0
            
            hit_rate = hit_rate + f
            print("\t\tCount #:",str(count).zfill(len(str(len(df_user)))),"Hit #:",str(f).zfill(len(str(len(df_user)))))
        
        hit_rate = hit_rate/len(df_user)
        hit_rates.append(hit_rate)
            

    N_counts.update({N:maths.array(hit_rates)})



For 10

	User # 0 User ID : 36
		Count #: 00 Hit #: 00
		Count #: 01 Hit #: 00
		Count #: 02 Hit #: 00
		Count #: 03 Hit #: 00
		Count #: 04 Hit #: 00
		Count #: 05 Hit #: 01
		Count #: 06 Hit #: 00
		Count #: 07 Hit #: 00
		Count #: 08 Hit #: 00
		Count #: 09 Hit #: 00
		Count #: 10 Hit #: 00
		Count #: 11 Hit #: 00
		Count #: 12 Hit #: 00
		Count #: 13 Hit #: 00
		Count #: 14 Hit #: 01
		Count #: 15 Hit #: 00
		Count #: 16 Hit #: 01
		Count #: 17 Hit #: 00
		Count #: 18 Hit #: 00
		Count #: 19 Hit #: 00

	User # 1 User ID : 34
		Count #: 00 Hit #: 00
		Count #: 01 Hit #: 00
		Count #: 02 Hit #: 00
		Count #: 03 Hit #: 01
		Count #: 04 Hit #: 00
		Count #: 05 Hit #: 00
		Count #: 06 Hit #: 01
		Count #: 07 Hit #: 00
		Count #: 08 Hit #: 00
		Count #: 09 Hit #: 00
		Count #: 10 Hit #: 00
		Count #: 11 Hit #: 00
		Count #: 12 Hit #: 00
		Count #: 13 Hit #: 00
		Count #: 14 Hit #: 00
		Count #: 15 Hit #: 00
		Count #: 16 Hit #: 00
		Count #: 17 Hit #: 01
		Count #: 18 Hit #: 00
		Count 

In [6]:
df_performance = analytics.DataFrame(N_counts)
df_performance

,10,15,20
0,0.15,0.25,0.30
1,0.20,0.25,0.20
2,0.05,0.25,0.30
3,0.15,0.20,0.25
4,0.15,0.30,0.20
5,0.20,0.30,0.05
6,0.05,0.40,0.20
7,0.40,0.30,0.55
8,0.25,0.30,0.30
9,0.20,0.20,0.30


In [7]:
df_performance.mean()

10    0.1675
15    0.2675
20    0.2775
dtype: float64